<a href="https://colab.research.google.com/github/oserikov/data-science-nlp/blob/master/1_%D0%BF%D1%80%D0%B5%D0%B4%D0%BE%D0%B1%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%BA%D0%B0_%D1%82%D0%B5%D0%BA%D1%81%D1%82%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -qq install yargy --progress-bar off
!pip -qq install pymorphy2 --progress-bar off
!pip -qq install rusenttokenize --progress-bar off
import nltk
nltk.download("punkt", quiet=True)
nltk.download("stopwords", quiet=True)

True

# Предобработка текста

1. **Токенизация и сегментация** &mdash; деление текста на слова и предложения.
2. **Нормализация** — приведение к одному регистру, удаляются пунктуации, исправление опечаток и т.д.
3. **Удаление стоп-слов** — слов, которые не несут никакой смысловой нагрузки (предлоги, союзы и т.п.) Список зависит от задачи!
4. 
    * **Стемминг** —  выделение псевдоосновы слова.
    * **Лемматизация** — приведение слов к словарной ("начальной") форме.

--- 
   
Дальше можно делать лингвистический анализ, но не всегда нужно. Весь пайплайн продолжается так:
5. **Part-of-Speech tagging (морфологическая разметка)** — приписывание частеречного тега или цепочки грамматических тегов (полный грамматический разбор) токену.
6. **Dependency parsing (парсинг зависимостей, синтаксическая разметка)** &mdash; нахождение синтаксических зависимостей между словами. Похоже на то, что в школе, с некоторыми отступлениями (e.g. подлежащее зависит от сказуемого) 
7. Семантическая и дискурсивная разметка (всё очень сыро в русском)

**NB!** Не всегда нужны все этапы, все зависит от задачи!

## Сегментация

**Сегментация** &mdash; разделение текста на предложения.

Сегментацию предложений иногда называют **сплиттингом**. 

Основные признаки — знаки препинания. "?", "!" как правило однозначны, проблемы возникают с "."  
Возможное решение: бинарный классификатор для сегментации предложений. Для каждой точки "." определить, является ли она концом предложения или нет. Ну и вообще машинное обучение, статистика и дистрибутивные свойства всякого в языке &mdash; сила.


In [0]:
text = """
Продаётся LADA 4x4. ПТС 01.12.2018, куплена 20 января 19 года, 10 000 км пробега. Комплектация полная. Новая в салоне 750 000, отдам за 650 000. Возможен обмен на ВАЗ-2110 или ВАЗ 2109 с вашей доплатой.

* Модификация: 1.6 MT (89 л.с.) 
* Владельцев по ПТС: 4+ 
* VIN или номер кузова: XTA21104*50****47 
* Мультимедиа и навигация: CD/DVD/Blu-ray 
* Шины и диски: 14" 

Краснодар, ул. Миклухо-Маклая, д. 4/5, подъезд 1 

Тел. 8(999)1234567, 8 903 987-65-43, +7 (351) 111 22 33 

e-mail: ivanov.ivan-61@mail.ru 

И.И. Иванов (Иван Иванович) 
"""

In [3]:
from nltk.tokenize import sent_tokenize

sents = sent_tokenize(text)
print(len(sents))
sents

11


['\nПродаётся LADA 4x4.',
 'ПТС 01.12.2018, куплена 20 января 19 года, 10 000 км пробега.',
 'Комплектация полная.',
 'Новая в салоне 750 000, отдам за 650 000.',
 'Возможен обмен на ВАЗ-2110 или ВАЗ 2109 с вашей доплатой.',
 '* Модификация: 1.6 MT (89 л.с.)',
 '* Владельцев по ПТС: 4+ \n* VIN или номер кузова: XTA21104*50****47 \n* Мультимедиа и навигация: CD/DVD/Blu-ray \n* Шины и диски: 14" \n\nКраснодар, ул.',
 'Миклухо-Маклая, д.',
 '4/5, подъезд 1 \n\nТел.',
 '8(999)1234567, 8 903 987-65-43, +7 (351) 111 22 33 \n\ne-mail: ivanov.ivan-61@mail.ru \n\nИ.И.',
 'Иванов (Иван Иванович)']

In [4]:
from rusenttokenize import ru_sent_tokenize

sents = ru_sent_tokenize(text)
print(len(sents))
sents

6


['Продаётся LADA 4x4.',
 'ПТС 01.12.2018, куплена 20 января 19 года, 10 000 км пробега.',
 'Комплектация полная.',
 'Новая в салоне 750 000, отдам за 650 000.',
 'Возможен обмен на ВАЗ-2110 или ВАЗ 2109 с вашей доплатой.',
 '* Модификация: 1.6 MT (89 л.с.) \n* Владельцев по ПТС: 4+ \n* VIN или номер кузова: XTA21104*50****47 \n* Мультимедиа и навигация: CD/DVD/Blu-ray \n* Шины и диски: 14" \n\nКраснодар, ул. Миклухо-Маклая, д. 4/5, подъезд 1 \n\nТел. 8(999)1234567, 8 903 987-65-43, +7 (351) 111 22 33 \n\ne-mail: ivanov.ivan-61@mail.ru \n\nИ.И. Иванов (Иван Иванович)']

In [22]:
from gensim.summarization.textcleaner import split_sentences

sents = split_sentences(text)
print(len(sents))
sents

18


['Продаётся LADA 4x4.',
 'ПТС 01.12.2018, куплена 20 января 19 года, 10 000 км пробега.',
 'Комплектация полная.',
 'Новая в салоне 750 000, отдам за 650 000.',
 'Возможен обмен на ВАЗ-2110 или ВАЗ 2109 с вашей доплатой.',
 '* Модификация: 1.6 MT (89 л.с.) ',
 '* Владельцев по ПТС: 4+ ',
 '* VIN или номер кузова: XTA21104*50****47 ',
 '* Мультимедиа и навигация: CD/DVD/Blu-ray ',
 '* Шины и диски: 14" ',
 'Краснодар, ул.',
 'Миклухо-Маклая, д.',
 '4/5, подъезд 1 ',
 'Тел.',
 '8(999)1234567, 8 903 987-65-43, +7 (351) 111 22 33 ',
 'e-mail: ivanov.ivan-61@mail.ru ',
 'И.И.',
 'Иванов (Иван Иванович) ']

Разные методы сработали по разному. Идеального решения обычно нет, всегда стоит посмотреть, какой вариант меньше всего ошибается на вашей задаче.

---
В этой строке я передаю привет тому, кто придумал назвать сегментирующий метод в nltk словом *tokenize*. Это отличный способ запутать всех, почему &mdash; смотрите дальше.


## Токенизация

**Сколько слов в этом предложении?**  
*На дворе трава, на траве дрова, не руби дрова на траве двора.*


* 12 словоформ: На, дворе, трава, на, траве, дрова, не, руби, дрова, на, траве, двора
* 6  лексем: на, не, двор, трава, дрова, рубить



Ниже будет несколько ссылок на Энциклопедию Кругосвет. Она классная ❤️.

### Лексема
**Лексема** &mdash; слово, рассматриваемое как единица словарного состава языка в совокупности всех его конкретных грамматических форм и выражающих их флексий, а также всех возможных значений (смысловых вариантов); **абстрактная двусторонняя единица словаря**. [Энциклопедия Кругосвет](https://www.krugosvet.ru/enc/lingvistika/leksema)  



### Двусторонний характер языкового знака
Языковой знак в лингвистике часто рассматривают как двухстороннюю сущность &mdash; пару $\left<\textit{означаемое}, \textit{означающее}\right>$. Автор идеи &mdash; Ф. де Соссюр.  [Энциклопедия Кругосвет](https://www.krugosvet.ru/enc/gumanitarnye_nauki/lingvistika/ZNAK.html)



### Словоформа
**Словоформа** &mdash; слово, рассматриваемое как представитель определенной лексемы и определенной грамматической формы. [Энциклопедия Кругосвет](https://www.krugosvet.ru/enc/gumanitarnye_nauki/lingvistika/SLOVOFORMA.html)  



### Токен $\neq$ слово


Интуитивно хочется поделить предложение на что-то типа слов, но вот в нашем объявлении и слова встречаются и цифры. В общем, всякое разное.  
Назовём эту размытую единицу предложения словом *токен* &mdash; оно вообще часто оказывается полезным для называния элементов упорядоченных последовательностей. 

Что в нашем тексте является токенами?

In [0]:
text = """
Продаётся LADA 4x4. ПТС 01.12.2018, куплена 20 января 19 года, 10 000 км пробега. Комплектация полная. Новая в салоне 750 000, отдам за 650 000. Возможен обмен на ВАЗ-2110 или ВАЗ 2109 с вашей доплатой.

* Модификация: 1.6 MT (89 л.с.) 
* Владельцев по ПТС: 4+ 
* VIN или номер кузова: XTA21104*50****47 
* Мультимедиа и навигация: CD/DVD/Blu-ray 
* Шины и диски: 14" 

Краснодар, ул. Миклухо-Маклая, д. 4/5, подъезд 1 

Тел. 8(999)1234567, 8 903 987-65-43, +7 (351) 111 22 33 

e-mail: ivanov.ivan-61@mail.ru 

И.И. Иванов (Иван Иванович) 
"""

Самая банальная токенизация: разбиение по пробелам

In [6]:
# самая банальная токенизация: разбиение по пробелам
tokens = text.split()
print(tokens)
len(tokens)

['Продаётся', 'LADA', '4x4.', 'ПТС', '01.12.2018,', 'куплена', '20', 'января', '19', 'года,', '10', '000', 'км', 'пробега.', 'Комплектация', 'полная.', 'Новая', 'в', 'салоне', '750', '000,', 'отдам', 'за', '650', '000.', 'Возможен', 'обмен', 'на', 'ВАЗ-2110', 'или', 'ВАЗ', '2109', 'с', 'вашей', 'доплатой.', '*', 'Модификация:', '1.6', 'MT', '(89', 'л.с.)', '*', 'Владельцев', 'по', 'ПТС:', '4+', '*', 'VIN', 'или', 'номер', 'кузова:', 'XTA21104*50****47', '*', 'Мультимедиа', 'и', 'навигация:', 'CD/DVD/Blu-ray', '*', 'Шины', 'и', 'диски:', '14"', 'Краснодар,', 'ул.', 'Миклухо-Маклая,', 'д.', '4/5,', 'подъезд', '1', 'Тел.', '8(999)1234567,', '8', '903', '987-65-43,', '+7', '(351)', '111', '22', '33', 'e-mail:', 'ivanov.ivan-61@mail.ru', 'И.И.', 'Иванов', '(Иван', 'Иванович)']


85

Есть другие всякие токенизаторы

In [7]:
# максимально разбивает
from yargy.tokenizer import MorphTokenizer

tokenizer = MorphTokenizer()
tokens = [_.value for _ in tokenizer(text)]
print(tokens)
len(tokens)

['\n', 'Продаётся', 'LADA', '4', 'x', '4', '.', 'ПТС', '01', '.', '12', '.', '2018', ',', 'куплена', '20', 'января', '19', 'года', ',', '10', '000', 'км', 'пробега', '.', 'Комплектация', 'полная', '.', 'Новая', 'в', 'салоне', '750', '000', ',', 'отдам', 'за', '650', '000', '.', 'Возможен', 'обмен', 'на', 'ВАЗ', '-', '2110', 'или', 'ВАЗ', '2109', 'с', 'вашей', 'доплатой', '.', '\n\n', '*', 'Модификация', ':', '1', '.', '6', 'MT', '(', '89', 'л', '.', 'с', '.', ')', '\n', '*', 'Владельцев', 'по', 'ПТС', ':', '4', '+', '\n', '*', 'VIN', 'или', 'номер', 'кузова', ':', 'XTA', '21104', '*', '50', '*', '*', '*', '*', '47', '\n', '*', 'Мультимедиа', 'и', 'навигация', ':', 'CD', '/', 'DVD', '/', 'Blu', '-', 'ray', '\n', '*', 'Шины', 'и', 'диски', ':', '14', '"', '\n\n', 'Краснодар', ',', 'ул', '.', 'Миклухо', '-', 'Маклая', ',', 'д', '.', '4', '/', '5', ',', 'подъезд', '1', '\n\n', 'Тел', '.', '8', '(', '999', ')', '1234567', ',', '8', '903', '987', '-', '65', '-', '43', ',', '+', '7', '(', '35

179

In [23]:
from nltk.tokenize import wordpunct_tokenize

tokens = wordpunct_tokenize(text)
print(tokens)
len(tokens)

['Продаётся', 'LADA', '4x4', '.', 'ПТС', '01', '.', '12', '.', '2018', ',', 'куплена', '20', 'января', '19', 'года', ',', '10', '000', 'км', 'пробега', '.', 'Комплектация', 'полная', '.', 'Новая', 'в', 'салоне', '750', '000', ',', 'отдам', 'за', '650', '000', '.', 'Возможен', 'обмен', 'на', 'ВАЗ', '-', '2110', 'или', 'ВАЗ', '2109', 'с', 'вашей', 'доплатой', '.', '*', 'Модификация', ':', '1', '.', '6', 'MT', '(', '89', 'л', '.', 'с', '.)', '*', 'Владельцев', 'по', 'ПТС', ':', '4', '+', '*', 'VIN', 'или', 'номер', 'кузова', ':', 'XTA21104', '*', '50', '****', '47', '*', 'Мультимедиа', 'и', 'навигация', ':', 'CD', '/', 'DVD', '/', 'Blu', '-', 'ray', '*', 'Шины', 'и', 'диски', ':', '14', '"', 'Краснодар', ',', 'ул', '.', 'Миклухо', '-', 'Маклая', ',', 'д', '.', '4', '/', '5', ',', 'подъезд', '1', 'Тел', '.', '8', '(', '999', ')', '1234567', ',', '8', '903', '987', '-', '65', '-', '43', ',', '+', '7', '(', '351', ')', '111', '22', '33', 'e', '-', 'mail', ':', 'ivanov', '.', 'ivan', '-', '61

161

In [8]:
from nltk.tokenize import word_tokenize

tokens = word_tokenize(text)
print(tokens)
len(tokens)

['Продаётся', 'LADA', '4x4', '.', 'ПТС', '01.12.2018', ',', 'куплена', '20', 'января', '19', 'года', ',', '10', '000', 'км', 'пробега', '.', 'Комплектация', 'полная', '.', 'Новая', 'в', 'салоне', '750', '000', ',', 'отдам', 'за', '650', '000', '.', 'Возможен', 'обмен', 'на', 'ВАЗ-2110', 'или', 'ВАЗ', '2109', 'с', 'вашей', 'доплатой', '.', '*', 'Модификация', ':', '1.6', 'MT', '(', '89', 'л.с', '.', ')', '*', 'Владельцев', 'по', 'ПТС', ':', '4+', '*', 'VIN', 'или', 'номер', 'кузова', ':', 'XTA21104*50****47', '*', 'Мультимедиа', 'и', 'навигация', ':', 'CD/DVD/Blu-ray', '*', 'Шины', 'и', 'диски', ':', '14', "''", 'Краснодар', ',', 'ул', '.', 'Миклухо-Маклая', ',', 'д', '.', '4/5', ',', 'подъезд', '1', 'Тел', '.', '8', '(', '999', ')', '1234567', ',', '8', '903', '987-65-43', ',', '+7', '(', '351', ')', '111', '22', '33', 'e-mail', ':', 'ivanov.ivan-61', '@', 'mail.ru', 'И.И', '.', 'Иванов', '(', 'Иван', 'Иванович', ')']


122

In [9]:
from nltk.tokenize import ToktokTokenizer

tokenizer = ToktokTokenizer()
tokens = tokenizer.tokenize(text)
print(tokens)
len(tokens)

['Продаётся', 'LADA', '4x4.', 'ПТС', '01.12.2018', ',', 'куплена', '20', 'января', '19', 'года', ',', '10', '000', 'км', 'пробега.', 'Комплектация', 'полная.', 'Новая', 'в', 'салоне', '750', '000', ',', 'отдам', 'за', '650', '000.', 'Возможен', 'обмен', 'на', 'ВАЗ-2110', 'или', 'ВАЗ', '2109', 'с', 'вашей', 'доплатой.', '*', 'Модификация', ':', '1.6', 'MT', '(', '89', 'л.с.', ')', '*', 'Владельцев', 'по', 'ПТС', ':', '4+', '*', 'VIN', 'или', 'номер', 'кузова', ':', 'XTA21104*50****47', '*', 'Мультимедиа', 'и', 'навигация', ':', 'CD/DVD/Blu-ray', '*', 'Шины', 'и', 'диски', ':', '14', '"', 'Краснодар', ',', 'ул.', 'Миклухо-Маклая', ',', 'д.', '4/5', ',', 'подъезд', '1', 'Тел.', '8(', '999', ')', '1234567', ',', '8', '903', '987-65-43', ',', '+7', '(', '351', ')', '111', '22', '33', 'e-mail', ':', 'ivanov.ivan-61@mail.ru', 'И.И.', 'Иванов', '(', 'Иван', 'Иванович', ')']


109

### Есть специальный токенизатор для твитов
он специально создан токенизировать твиты.

In [10]:
# специальный токенизатор для твитов
from nltk.tokenize import TweetTokenizer

tknzr = TweetTokenizer()
tweet = "@remy This is a cooool #dummysmiley: :-) :-P <3 and some arrows < > -> <--"
tknzr.tokenize(tweet)

['@remy',
 'This',
 'is',
 'a',
 'cooool',
 '#dummysmiley',
 ':',
 ':-)',
 ':-P',
 '<3',
 'and',
 'some',
 'arrows',
 '<',
 '>',
 '->',
 '<--']

## Нормализация
* удаление пунктуации
* преобразование регистра
* исправление опечаток
* удаление стоп-слов

### Удаление пунктуации

In [0]:
text = """
Продаётся LADA 4x4. ПТС 01.12.2018, куплена 20 января 19 года, 10 000 км пробега. Комплектация полная. Новая в салоне 750 000, отдам за 650 000. Возможен обмен на ВАЗ-2110 или ВАЗ 2109 с вашей доплатой.

* Модификация: 1.6 MT (89 л.с.) 
* Владельцев по ПТС: 4+ 
* VIN или номер кузова: XTA21104*50****47 
* Мультимедиа и навигация: CD/DVD/Blu-ray 
* Шины и диски: 14" 

Краснодар, ул. Миклухо-Маклая, д. 4/5, подъезд 1 

Тел. 8(999)1234567, 8 903 987-65-43, +7 (351) 111 22 33 

e-mail: ivanov.ivan-61@mail.ru 

И.И. Иванов (Иван Иванович) 
"""

В `string.punctuation` хранятся строкой популярные знаки препинания. Строку уместно дополнять по смыслу задачи :)

In [12]:
import string
print("символы пунктуации: ", string.punctuation)

clean_words = [w.strip(string.punctuation) for w in word_tokenize(text)]
clean_words = [w for w in clean_words if w]
print(clean_words)

символы пунктуации:  !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['Продаётся', 'LADA', '4x4', 'ПТС', '01.12.2018', 'куплена', '20', 'января', '19', 'года', '10', '000', 'км', 'пробега', 'Комплектация', 'полная', 'Новая', 'в', 'салоне', '750', '000', 'отдам', 'за', '650', '000', 'Возможен', 'обмен', 'на', 'ВАЗ-2110', 'или', 'ВАЗ', '2109', 'с', 'вашей', 'доплатой', 'Модификация', '1.6', 'MT', '89', 'л.с', 'Владельцев', 'по', 'ПТС', '4', 'VIN', 'или', 'номер', 'кузова', 'XTA21104*50****47', 'Мультимедиа', 'и', 'навигация', 'CD/DVD/Blu-ray', 'Шины', 'и', 'диски', '14', 'Краснодар', 'ул', 'Миклухо-Маклая', 'д', '4/5', 'подъезд', '1', 'Тел', '8', '999', '1234567', '8', '903', '987-65-43', '7', '351', '111', '22', '33', 'e-mail', 'ivanov.ivan-61', 'mail.ru', 'И.И', 'Иванов', 'Иван', 'Иванович']


Проверим, нет ли у нас какой-то неутённой пунктуации (но вроде нет (но проверим))

In [28]:
used_nonalnum_characters = {c for c in text if not c.isalnum()}
print(used_nonalnum_characters.difference(string.punctuation))

{'\n', ' '}


### Преобразование регистра

In [0]:
text = """
Продаётся LADA 4x4. ПТС 01.12.2018, куплена 20 января 19 года, 10 000 км пробега. Комплектация полная. Новая в салоне 750 000, отдам за 650 000. Возможен обмен на ВАЗ-2110 или ВАЗ 2109 с вашей доплатой.

* Модификация: 1.6 MT (89 л.с.) 
* Владельцев по ПТС: 4+ 
* VIN или номер кузова: XTA21104*50****47 
* Мультимедиа и навигация: CD/DVD/Blu-ray 
* Шины и диски: 14" 

Краснодар, ул. Миклухо-Маклая, д. 4/5, подъезд 1 

Тел. 8(999)1234567, 8 903 987-65-43, +7 (351) 111 22 33 

e-mail: ivanov.ivan-61@mail.ru 

И.И. Иванов (Иван Иванович) 
"""

In [14]:
clean_words = [w.lower() for w in clean_words if w]
print(clean_words)

['продаётся', 'lada', '4x4', 'птс', '01.12.2018', 'куплена', '20', 'января', '19', 'года', '10', '000', 'км', 'пробега', 'комплектация', 'полная', 'новая', 'в', 'салоне', '750', '000', 'отдам', 'за', '650', '000', 'возможен', 'обмен', 'на', 'ваз-2110', 'или', 'ваз', '2109', 'с', 'вашей', 'доплатой', 'модификация', '1.6', 'mt', '89', 'л.с', 'владельцев', 'по', 'птс', '4', 'vin', 'или', 'номер', 'кузова', 'xta21104*50****47', 'мультимедиа', 'и', 'навигация', 'cd/dvd/blu-ray', 'шины', 'и', 'диски', '14', 'краснодар', 'ул', 'миклухо-маклая', 'д', '4/5', 'подъезд', '1', 'тел', '8', '999', '1234567', '8', '903', '987-65-43', '7', '351', '111', '22', '33', 'e-mail', 'ivanov.ivan-61', 'mail.ru', 'и.и', 'иванов', 'иван', 'иванович']


### Исправление опечаток

Как правило, спеллчекеры основаны на **расстоянии Левенштейна** (редакционное расстояние, edit distance). Это минимальное количество операций вставки одного символа, удаления одного символа и замены одного символа на другой, необходимых для превращения одной строки в другую. Модификация этого алгоритма — расстояние Дамерау-Левенштейна — включает также операцию перестановки символов.

Для специфических задач e.g. исправление опечаток в фамилиях или названиях городов может оказаться полезным Metaphone &mdash; *фонетический* алгоритм группирования символов по схожести звучания. 
* [Простейший спеллчекер Норвига](https://norvig.com/spell-correct.html)
* [Hunspell](http://hunspell.github.io/)
* [JamSpell](https://github.com/bakwc/JamSpell)
* [Yandex Speller API](https://tech.yandex.ru/speller/doc/dg/concepts/api-overview-docpage/)
* [Статья о фонетических алгоритмах на хабре](https://habr.com/ru/post/114947/)
* etc. etc.

## Удаление стоп-слов

**Стоп-слова** — высокочастотные слова, которые не дают нам никакой информации о конкретном тексте. Они составляют верхушку частотного списка в любом языке. Набор стоп-слов не универсален, он будет зависеть от вашей задачи!

В NLTK есть готовые списки стоп-слов для многих языков. Хороший стписок стоп-слов есть у яндекс.директа, на его основе составлен [список стоп-слов для анализа художественных текстов](./data/rus_stopwords.txt) [@ancatmara](https://github.com/ancatmara).

In [15]:
from nltk.corpus import stopwords

# смотрим, какие языки есть
print(stopwords.fileids())

['arabic', 'azerbaijani', 'danish', 'dutch', 'english', 'finnish', 'french', 'german', 'greek', 'hungarian', 'indonesian', 'italian', 'kazakh', 'nepali', 'norwegian', 'portuguese', 'romanian', 'russian', 'slovene', 'spanish', 'swedish', 'tajik', 'turkish']


In [16]:
russian_stopwords = stopwords.words('russian')
print(russian_stopwords)

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впр

In [0]:
clean_words = [w for w in clean_words if w not in russian_stopwords]

## Стемминг

**Стемминг** — отсечение от слова окончаний и суффиксов, чтобы оставшаяся часть, называемая stem, была одинаковой для всех грамматических форм слова. Стем необязательно совпадает с морфлогической основой слова. Одинаковый стем может получиться и не у однокоренных слов и наоборот — в этом проблема стемминга. 

* 1-ый вид ошибки: белый, белка, белье $\rightarrow$  бел

* 2-ой вид ошибки: трудность, трудный $\rightarrow$  трудност, труд 

* 3-ий вид ошибки: быстрый, быстрее $\rightarrow$  быст, побыстрее $\rightarrow$  побыст


### Алгоритм Портера
Самый простой алгоритм стемминга, алгоритм Портера, состоит из 5 циклов команд, на каждом цикле – операция удаления / замены суффикса. Возможны вероятностные расширения алгоритма.

### Snowball stemmer
Улучшенный вариант стеммера Портера; в отличие от него умеет работать не только с английским текстом.

In [18]:
from nltk.stem.snowball import SnowballStemmer

print(SnowballStemmer.languages)

('arabic', 'danish', 'dutch', 'english', 'finnish', 'french', 'german', 'hungarian', 'italian', 'norwegian', 'porter', 'portuguese', 'romanian', 'russian', 'spanish', 'swedish')


In [34]:
snowball = SnowballStemmer("russian")
import pandas as pd

pd.DataFrame([(w, snowball.stem(w)) for w in clean_words], 
             columns=("wordform", "stem")).head(20)
    

,wordform,stem
0,продаётся,прода
1,lada,lad
2,4x4,4x4
3,птс,птс
4,01.12.2018,01.12.2018
5,куплена,купл
6,20,20
7,января,январ
8,19,19
9,года,год


## Лемматизация

**Лемматизация** — процесс приведения словоформы к нормальной (словарной) форме. Это более сложная задача, чем стемминг, но и результаты дает гораздо более осмысленные, особенно для языков с богатой морфологией.

* кошке, кошку, кошкам, кошкой $\rightarrow$ кошка
* бежал, бежит, бегу $\rightarrow$  бежать
* белому, белым, белыми $\rightarrow$ белый


####  pymorphy2

**pymorphy2** &mdash; это полноценный морфологический анализатор для русского языка, целиком написанный на Python. Ещё умеет ставить слова в нужную форму.

In [33]:
from pymorphy2 import MorphAnalyzer

morph_analyzer = MorphAnalyzer()

pd.DataFrame([(w, morph_analyzer.parse(w)[0].normal_form) for w in clean_words],
             columns=("wordform", "lemma"))


,wordform,lemma
0,продаётся,продаваться
1,lada,lada
2,4x4,4x4
3,птс,птс
4,01.12.2018,01.12.2018
...,...,...
69,mail.ru,mail.ru
70,и.и,и.и
71,иванов,иванов
72,иван,иван
